# Overview and references

You will search several remote biological databases for gene sequences, annotations, and relationships between genes. You will then import the data you’ve found into a database that you will design.


For this lab, you will import data of a particular type (e.g., a gene sequence, annotation, or pathway) into an appropriated-designed table and perform table joins that link together the data you’ve retrieved.


You are expected to keep a thorough record of everything you did in your notebook. Any data that you’ve pulled from a web browser should be noted in this notebook. Copy and paste any terminal commands you used into a Markdown section and explain what the input was, what the tool did, and what the output was.


### SQLite
https://www.tutorialspoint.com/sqlite/index.htm https://docs.python.org/3/library/sqlite3.html

### Relational Databases
https://en.wikipedia.org/wiki/Relational_database https://mariadb.com/kb/en/library/relational-databases-basic-terms/ https://en.wikipedia.org/wiki/Associative_entity

### BioPython
https://biopython.org/DIST/docs/api/Bio.Entrez-module.html

### Pandas
https://pandas.pydata.org/pandas-docs/stable/index.html

### Biological Databases
https://genome.ucsc.edu/ https://www.ncbi.nlm.nih.gov/search/ https://reactome.org/ https://pfam.xfam.org/ https://www.genome.jp/kegg/

# Background 

The lab you’re working in studies metabolism. Over the years, many students and post-docs have studied enzymes from organisms across the tree of life, from flies, worms, mice, and E. coli. Your PI wants you to build a database of these enzymes that future researchers in the lab can use going forward.


# Database Design

For this assignment, we’ll focus on three pathways: 

1. glycolysis
2. the citric acid cycle
3. the pentose phosphate pathway. 

We will consider genes in these pathways from 

1. Drosophila melanogaster
2. Homo sapiens 

And we will construct the following tables:

**Pathway Table**. Should include at least name and description. What fields should be included to enable joins with other tables?

**Enzyme Table**. Should include at least name, function, and enzyme commission (EC) number. What fields should be included to enable joins with other tables?

**Gene Table**. Should include at least name, description, organism, and nucleotide sequence. What fields should be included to enable joins with other tables?

# Visualization
Pandas provides a useful python interface for working with data tables. In this lab, we will use pandas DataFrames for visualization only. The rest of the lab should have been completed using SQL (and python interface to it). Read your SQL tables using the [`pandas.read_sql`](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) method.

# Part 1: Database/table creation


Now that your tables are designed, try creating your database by using the function below. You might have to delete and re-create your database (the file) if mistakes are made and perform one final execution with the correct code.

The command can be executed like this:

```python
command = """
    CREATE TABLE <fill in your code>
"""
execute_sql_command(command)
```

In [1]:
%%capture
import sqlite3
import pandas as pd
!conda install -y -q -c conda-forge biopython

In [2]:
def execute_sql_command(command: str, database_name: str = "lab3.db") -> None:
    """
    Function to execute a SQL command from Python.
    
    Parameters
    ----------
    command: str
        SQL command (use strings with three quotes on each side
        so that it can be a multiline string
    database_name: str
        File name of the database (e.g, "my.db")
    
    Returns
    -------
    No return, executes the command
    """
    
    # will create if not present
    conn = sqlite3.connect(database_name)
    c = conn.cursor()
    
    c.execute(command)
    # saves the changes
    conn.commit()

### Part 1.1: Create pathways table

In [3]:
command = """
    CREATE TABLE pathways
    (name TEXT PRIMARY KEY,
    description TEXT)
    
"""
execute_sql_command(command)

### Part 1.2: Create enzymes table

In [4]:
command = """
    CREATE TABLE enzymes
    (name TEXT,
    description TEXT,
    ecnum TEXT PRIMARY KEY,
    pathway TEXT)
    
"""
execute_sql_command(command)

### Part 1.3: Create genes table

In [5]:
command = """
    CREATE TABLE genes
    (name TEXT,
    description TEXT,
    organism TEXT,
    nuc_seq TEXT,
    enzyme_ecnum TEXT) 
"""

execute_sql_command(command)

# Searching databases and compiling data

A good place to start will be this map of glycolysis from KEGG: 

https://www.genome.jp/kegg-bin/show_pathway?map00010
    
There are a lot of genes in these pathways and how thoroughly you curate your database is up to you, but for the sake of time:

1. Pick 4 enzymes from each of glycolysis, TCA cycle, and pentose phosphate. These enzymes should be relevant for both species. Look for "change pathway type" in the Kegg pathway map.
2. Inspect information and sequences from Drosophila and human for those enzymes in KEGG database (no need to add anything in this notebook for this step, just inspect for yourself).

# INSERTing your data into your database

- In your pathways table, you should have a row per pathway (3 in total).
- In your enzymes table, you should have one row per enzyme (12 in total).
- In your gene table, you’ll be creating `4 enzymes * 3 pathways * 2 organisms = 24 rows`.

# Part 2: populate pathways table
For each pathway, insert a row into the database using the `INSERT` operation. Once you are done, print the contents of the table using `SELECT` and `pd.read_sql`.

In [6]:
#Source: https://www.genome.jp/pathway/map00010
insert1 = """
    INSERT INTO pathways
    VALUES
    ('glycolysis',
    'Glycolysis is the process of converting glucose into pyruvate and generating small amounts of ATP (energy) and NADH (reducing power).')
"""

#Source: https://www.genome.jp/pathway/map00020
insert2 = """
    INSERT INTO pathways
    VALUES
    ('citric acid cycle',
    'The citrate cycle (TCA cycle, Krebs cycle) is an important aerobic pathway for the final steps of the oxidation of carbohydrates and fatty acids.')
"""

#Source: https://www.genome.jp/pathway/map00030
insert3 = """
    INSERT INTO pathways
    VALUES
    ('pentose phosphate pathway',
    'The pentose phosphate pathway is a process of glucose turnover that produces NADPH as reducing equivalents and pentoses as essential parts of nucleotides.')
"""

commands = [insert1, insert2, insert3]
for command in commands: execute_sql_command(command)

In [7]:
conn = sqlite3.connect('lab3.db')
pd.read_sql('SELECT * FROM pathways', conn)

,name,description
0,glycolysis,Glycolysis is the process of converting glucos...
1,citric acid cycle,"The citrate cycle (TCA cycle, Krebs cycle) is ..."
2,pentose phosphate pathway,The pentose phosphate pathway is a process of ...


# Part 3: populate enzymes table
Similarly, populate the enzymes table and print its contents.

In [8]:
#SOURCE: https://www.genome.jp/entry/3.1.3.9
insert1 = """
    INSERT INTO enzymes
    VALUES
    ('glucose-6-phosphatase',
    'Wide distribution in animal tissues.',
    '3.1.3.9',
    'glycolysis')
"""

#SOURCE: https://www.genome.jp/entry/5.4.2.2
insert2 = """
    INSERT INTO enzymes
    VALUES
    ('phosphoglucomutase',
    'Maximum activity is only obtained in the presence of alpha-D-glucose 1,6-bisphosphate.',
    '5.4.2.2',
    'glycolysis')
"""

#SOURCE: https://www.genome.jp/entry/2.7.1.1
insert3 = """
    INSERT INTO enzymes
    VALUES
    ('hexokinase',
    'D-Glucose, D-mannose, D-fructose, sorbitol and D-glucosamine can act as acceptors; ITP and dATP can act as donors.',
    '2.7.1.1',
    'glycolysis')
"""

#SOURCE: https://www.genome.jp/entry/5.1.3.3
insert4 = """
    INSERT INTO enzymes
    VALUES
    ('aldose 1-epimerase',
    'This enzyme catalyses the first step in galactose metabolism.',
    '5.1.3.3',
    'glycolysis')
"""

#SOURCE: https://www.genome.jp/entry/4.1.1.32
insert5 = """
    INSERT INTO enzymes
    VALUES
    ('phosphoenolpyruvate carboxykinase (GTP)',
    'ITP can act as phosphate donor.',
    '4.1.1.32',
    'citric acid cycle')
"""

#SOURCE: https://www.genome.jp/entry/2.3.1.12
insert6 = """
    INSERT INTO enzymes
    VALUES
    ('dihydrolipoyllysine-residue acetyltransferase',
    'Belongs to class: Transferring groups other than aminoacyl groups',
    '2.3.1.12',
    'citric acid cycle')
"""

#SOURCE: https://www.genome.jp/entry/1.2.4.1
insert7 = """
    INSERT INTO enzymes
    VALUES
    ('pyruvate dehydrogenase (acetyl-transferring)',
    'Contains thiamine diphosphate.',
    '1.2.4.1',
    'citric acid cycle')
"""

#SOURCE: https://www.genome.jp/entry/4.2.1.3
insert8 = """
    INSERT INTO enzymes
    VALUES
    ('aconitate hydratase',
    'Besides interconverting citrate and cis-aconitate, it also interconverts cis-aconitate with isocitrate and, hence, interconverts citrate and isocitrate.',
    '4.2.1.3',
    'citric acid cycle')
"""

#SOURCE: https://www.genome.jp/entry/1.1.1.49
insert9 = """
    INSERT INTO enzymes
    VALUES
    ('glucose-6-phosphate dehydrogenase (NADP+)',
    'The enzyme catalyses a step of the pentose phosphate pathway.',
    '1.1.1.49',
    'pentose phosphate pathway')
"""

#SOURCE: https://www.genome.jp/entry/5.3.1.9
insert10 = """
    INSERT INTO enzymes
    VALUES
    ('glucose-6-phosphate isomerase',
    'Also catalyses the anomerization of D-glucose 6-phosphate.',
    '5.3.1.9',
    'pentose phosphate pathway')
"""

#SOURCE: https://www.genome.jp/entry/3.1.1.31
insert11 = """
    INSERT INTO enzymes
    VALUES
    ('6-phosphogluconolactonase',
    'Belongs to the class: Carboxylic-ester hydrolases',
    '3.1.1.31',
    'pentose phosphate pathway')
"""

#SOURCE: https://www.genome.jp/entry/5.1.3.1
insert12 = """
    INSERT INTO enzymes
    VALUES
    ('ribulose-phosphate 3-epimerase',
    'The enzyme also converts D-erythrose 4-phosphate into D-erythrulose 4-phosphate and D-threose 4-phosphate.',
    '5.1.3.1',
    'pentose phosphate pathway')
"""

commands = [insert1, insert2, insert3, insert4, insert5, insert6, insert7, insert8, insert9, insert10, insert11, insert12]
for command in commands: execute_sql_command(command)

In [9]:
conn = sqlite3.connect('lab3.db')
pd.read_sql('SELECT * FROM enzymes', conn)

,name,description,ecnum,pathway
0,glucose-6-phosphatase,Wide distribution in animal tissues.,3.1.3.9,glycolysis
1,phosphoglucomutase,Maximum activity is only obtained in the prese...,5.4.2.2,glycolysis
2,hexokinase,"D-Glucose, D-mannose, D-fructose, sorbitol and...",2.7.1.1,glycolysis
3,aldose 1-epimerase,This enzyme catalyses the first step in galact...,5.1.3.3,glycolysis
4,phosphoenolpyruvate carboxykinase (GTP),ITP can act as phosphate donor.,4.1.1.32,citric acid cycle
5,dihydrolipoyllysine-residue acetyltransferase,Belongs to class: Transferring groups other th...,2.3.1.12,citric acid cycle
6,pyruvate dehydrogenase (acetyl-transferring),Contains thiamine diphosphate.,1.2.4.1,citric acid cycle
7,aconitate hydratase,Besides interconverting citrate and cis-aconit...,4.2.1.3,citric acid cycle
8,glucose-6-phosphate dehydrogenase (NADP+),The enzyme catalyses a step of the pentose pho...,1.1.1.49,pentose phosphate pathway
9,glucose-6-phosphate isomerase,Also catalyses the anomerization of D-glucose ...,5.3.1.9,pentose phosphate pathway


# Part 4: retrieve gene information

For genes, you will now automate this process using `Bio.Entrez`, which can be used to pull entire annotations for each enzyme from each organism—and more. You will ask it to return data as `GenBank` and later will use `Bio.SeqIO` to read the written `GenBank` file.

For this part, complete the commented code in the function below.

In [10]:
from Bio import Entrez
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

Entrez.email = 'travishanly@berkeley.edu'

def save_enzyme(ec_number: str, organism: str, file: str) -> None:
    """
    Pull and save most relevant annotation for enzyme/organism pair
    
    Parameters
    ----------
    ec_number: str
        Number assigned by the Enzyme Commission to designate a particular enzyme 
        or by the Chemical Abstracts Service (CAS) for Registry Numbers. 
    organism: str
        Organism name
    file: str
        Name of saved file. SHOULD HAVE .gb extension
        Points will be deducted if these files are committed to the repo.
    
    Returns
    -------
    No return, executes the command
    """
    term = ec_number  + " AND " + organism + "[Organism] AND 1:10000[SLEN]"
    handle = Entrez.esearch(db = 'nucleotide',
                           term = term ,
                           sort = 'relevance',
                           idtype = 'acc')
    for i in Entrez.read(handle)['IdList']:
        handle = Entrez.efetch(db = 'nucleotide', id = i, rettype = 'gb', retmode = 'text', retmax=1)
        seq = SeqIO.read(handle, 'genbank')
        if seq.annotations['organism'] == organism:
            # write the sequence to file using `SeqIO.write` in genbank (.gb) format.
            SeqIO.write(seq, file, "genbank")
            return

Run `save_enzyme` 24 times (`4 enzymes * 3 pathways * 2 organisms = 24`). This can take a while.

In [11]:
conn = sqlite3.connect('lab3.db')
enzyme_numbers = list(pd.read_sql('SELECT ecnum FROM enzymes', conn)['ecnum'])
organisms = ["Homo sapiens", "Drosophila melanogaster"]

for organism in organisms:
    for ecnum in enzyme_numbers:
        file = organism + "_" + ecnum.replace(".", "-") + ".gb"
        save_enzyme(ecnum, organism, file)

# Part 5: populate genes table

Since we managed to pull the data directly into `genbank` files using `Bio.Entrez`, a for loop over the files can be used to insert your data into your table (make sure to use `execute_sql_command`). Print the table contents.

In [12]:
enzyme_numbers = list(pd.read_sql('SELECT ecnum FROM enzymes', conn)['ecnum'])
organisms = ["Homo sapiens", "Drosophila melanogaster"]

for organism in organisms:
    for ecnum in enzyme_numbers:
        file = organism + "_" + ecnum.replace(".", "-") + ".gb"
        seq = SeqIO.read(file, "genbank")
        command = f"""INSERT INTO genes
                VALUES
                ('{seq.id}',
                '{seq.description.replace("'", "")}',
                '{organism}',
                '{seq.seq}',
                '{ecnum}')"""
        execute_sql_command(command)       

In [13]:
conn = sqlite3.connect('lab3.db')
pd.read_sql('SELECT * FROM genes', conn)

,name,description,organism,nuc_seq,enzyme_ecnum
0,AK302341.1,"Homo sapiens cDNA FLJ56794 complete cds, highl...",Homo sapiens,AAAACGGTCGTACACTTCGGGGCTGCGAGCGCGGAGGGCGACGACG...,1.1.1.49
1,AK296457.1,"Homo sapiens cDNA FLJ52314 complete cds, highl...",Homo sapiens,ACCCGCGTCGTGCCTCCTGGGTTGTGAGGAGTCGCCGCTGCCGCCA...,1.2.4.1
2,AK299562.1,"Homo sapiens cDNA FLJ51063 complete cds, highl...",Homo sapiens,CCCGTTCGTCGCAACAGCGTGACTACAGGGTATGGCGGGGTCCGGG...,2.3.1.12
3,AK294428.1,"Homo sapiens cDNA FLJ56506 complete cds, highl...",Homo sapiens,GAGGAGCCGCCGAGCAGCCGCCGGAGGACCACGGCTCGCCAGGGCT...,2.7.1.1
4,CD580465.2,EST_PSF016769 hEx1 (RZPD no. 800) Homo sapiens...,Homo sapiens,CTCCCCGCCGCCGCCCTCGCCATGGCCGCGCCGGCCCCGGGCCTCA...,3.1.1.31
5,AK303771.1,"Homo sapiens cDNA FLJ51443 complete cds, highl...",Homo sapiens,ATAGCAGAGCAATCACCACCAAGCCTGGAATAACTGCAAGGGCTCT...,3.1.3.9
6,AK316206.1,"Homo sapiens cDNA, FLJ79105 complete cds, high...",Homo sapiens,GTGGCTCGCTTCGCCGCGCTCCCTCCTTCCCCGCCTTCCATACCTC...,4.1.1.32
7,AK302691.1,"Homo sapiens cDNA FLJ51705 complete cds, highl...",Homo sapiens,ATGCGACCTCATCTTTGTCAGTGCACAAAATGGCGCCCTACAGCCT...,4.2.1.3
8,AK303184.1,"Homo sapiens cDNA FLJ52962 complete cds, highl...",Homo sapiens,AACTTGCTTTTGGGAGCCAGCGGTATGGCGTCGGGCTGCAAGATTG...,5.1.3.1
9,NM_138801.3,"Homo sapiens galactose mutarotase (GALM), mRNA",Homo sapiens,GCTGGAGTTTGAAGAGCGGGCAGTGGCTGCACACGCCAAACTTTCC...,5.1.3.3


# Part 6: Query data with JOIN

You should have added fields to your tables to link data together. For this part of the lab we would like to retrieve information from the three tables and print it with `pd.read_sql`.

### Part 6.1
Make a join of the three tables and display the results.

In [14]:
conn = sqlite3.connect('lab3.db')
select_command = """
    SELECT genes.name AS gene, genes.description AS gene_description, organism, nuc_seq, 
    enzymes.name AS enzyme, enzymes.description AS enzyme_description, ecnum, 
    pathway, pathways.description AS pathway_description  
    FROM genes
    LEFT JOIN enzymes ON genes.enzyme_ecnum = enzymes.ecnum
    LEFT JOIN pathways ON enzymes.pathway = pathways.name
"""

pd.read_sql(select_command, conn)

,gene,gene_description,organism,nuc_seq,enzyme,enzyme_description,ecnum,pathway,pathway_description
0,AK302341.1,"Homo sapiens cDNA FLJ56794 complete cds, highl...",Homo sapiens,AAAACGGTCGTACACTTCGGGGCTGCGAGCGCGGAGGGCGACGACG...,glucose-6-phosphate dehydrogenase (NADP+),The enzyme catalyses a step of the pentose pho...,1.1.1.49,pentose phosphate pathway,The pentose phosphate pathway is a process of ...
1,AK296457.1,"Homo sapiens cDNA FLJ52314 complete cds, highl...",Homo sapiens,ACCCGCGTCGTGCCTCCTGGGTTGTGAGGAGTCGCCGCTGCCGCCA...,pyruvate dehydrogenase (acetyl-transferring),Contains thiamine diphosphate.,1.2.4.1,citric acid cycle,"The citrate cycle (TCA cycle, Krebs cycle) is ..."
2,AK299562.1,"Homo sapiens cDNA FLJ51063 complete cds, highl...",Homo sapiens,CCCGTTCGTCGCAACAGCGTGACTACAGGGTATGGCGGGGTCCGGG...,dihydrolipoyllysine-residue acetyltransferase,Belongs to class: Transferring groups other th...,2.3.1.12,citric acid cycle,"The citrate cycle (TCA cycle, Krebs cycle) is ..."
3,AK294428.1,"Homo sapiens cDNA FLJ56506 complete cds, highl...",Homo sapiens,GAGGAGCCGCCGAGCAGCCGCCGGAGGACCACGGCTCGCCAGGGCT...,hexokinase,"D-Glucose, D-mannose, D-fructose, sorbitol and...",2.7.1.1,glycolysis,Glycolysis is the process of converting glucos...
4,CD580465.2,EST_PSF016769 hEx1 (RZPD no. 800) Homo sapiens...,Homo sapiens,CTCCCCGCCGCCGCCCTCGCCATGGCCGCGCCGGCCCCGGGCCTCA...,6-phosphogluconolactonase,Belongs to the class: Carboxylic-ester hydrolases,3.1.1.31,pentose phosphate pathway,The pentose phosphate pathway is a process of ...
5,AK303771.1,"Homo sapiens cDNA FLJ51443 complete cds, highl...",Homo sapiens,ATAGCAGAGCAATCACCACCAAGCCTGGAATAACTGCAAGGGCTCT...,glucose-6-phosphatase,Wide distribution in animal tissues.,3.1.3.9,glycolysis,Glycolysis is the process of converting glucos...
6,AK316206.1,"Homo sapiens cDNA, FLJ79105 complete cds, high...",Homo sapiens,GTGGCTCGCTTCGCCGCGCTCCCTCCTTCCCCGCCTTCCATACCTC...,phosphoenolpyruvate carboxykinase (GTP),ITP can act as phosphate donor.,4.1.1.32,citric acid cycle,"The citrate cycle (TCA cycle, Krebs cycle) is ..."
7,AK302691.1,"Homo sapiens cDNA FLJ51705 complete cds, highl...",Homo sapiens,ATGCGACCTCATCTTTGTCAGTGCACAAAATGGCGCCCTACAGCCT...,aconitate hydratase,Besides interconverting citrate and cis-aconit...,4.2.1.3,citric acid cycle,"The citrate cycle (TCA cycle, Krebs cycle) is ..."
8,AK303184.1,"Homo sapiens cDNA FLJ52962 complete cds, highl...",Homo sapiens,AACTTGCTTTTGGGAGCCAGCGGTATGGCGTCGGGCTGCAAGATTG...,ribulose-phosphate 3-epimerase,The enzyme also converts D-erythrose 4-phospha...,5.1.3.1,pentose phosphate pathway,The pentose phosphate pathway is a process of ...
9,NM_138801.3,"Homo sapiens galactose mutarotase (GALM), mRNA",Homo sapiens,GCTGGAGTTTGAAGAGCGGGCAGTGGCTGCACACGCCAAACTTTCC...,aldose 1-epimerase,This enzyme catalyses the first step in galact...,5.1.3.3,glycolysis,Glycolysis is the process of converting glucos...


### Part 6.2
Retrieve just the description of all the genes associated with glycolisis.

In [18]:
conn = sqlite3.connect('lab3.db')
select_command = """
    SELECT genes.description AS gene_description 
    FROM genes
    LEFT JOIN enzymes ON genes.enzyme_ecnum = enzymes.ecnum
    LEFT JOIN pathways ON enzymes.pathway = pathways.name
    WHERE pathway = 'glycolysis'
"""

pd.read_sql(select_command, conn)

,gene_description
0,"Homo sapiens cDNA FLJ56506 complete cds, highl..."
1,"Homo sapiens cDNA FLJ51443 complete cds, highl..."
2,"Homo sapiens galactose mutarotase (GALM), mRNA"
3,"Homo sapiens cDNA FLJ50663 complete cds, highl..."
4,"Drosophila melanogaster hexokinase C (Hex-C), ..."
5,Drosophila melanogaster Glucose-6-Phosphatase ...
6,Drosophila melanogaster uncharacterized protei...
7,Drosophila melanogaster phosphoglucose mutase ...


### Part 6.3
Are the joins one-to-one, many-to-one, or one-to-many?

### Answer

The joins are many-to-one.

We start with the genes tables as the 'backbone' with 24 rows, then execute a 2-to-1 join to bring in the genes data, followed by an 8-to-1 join to bring in the pathways data.

# Submission

1. Commit and push this notebook,
2. Git add and commit the database file (then push)
3. Submit repo link on GitHub